In [1]:
import os
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa


AUDIO_FOLDER  = './audio/'

In [2]:
# FMA: A Dataset For Music Analysis
# Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.
# Retrieved from https://github.com/mdeff/fma -> features.py

def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()

In [3]:
# FMA: A Dataset For Music Analysis
# Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.
# Retrieved from https://github.com/mdeff/fma -> features.py

def compute_features(tid,filepath):

    features = pd.Series(index=columns(), dtype=np.float32, name=tid)

    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rmse(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}: {}'.format(tid, repr(e)))

    return features

In [4]:
# Function that return every mp3 files in the AUDIO_FOLDER and nothing else (matters :P)
def mp3_finder(path):
    audio_files = list();
    for root, dirs, files in os.walk(path):
        for f in files:
            filename = os.path.join(root, f)
            if filename.endswith('.mp3'):
                audio_files.append(filename)
                
    return audio_files

In [6]:
# Creation of a list of Series (returned from compute_features)
# Creation of a DF that link the track_id with the file name just in case

listOfSeries = list()
link_df = pd.DataFrame(columns=['Track ID', 'Song name'])

for k, file in enumerate(mp3_finder(AUDIO_FOLDER)):
    link_df = link_df.append({'Track ID': -k-1, 'Song name': os.path.splitext(os.path.basename(file))[0]}, ignore_index=True)
    listOfSeries.append(compute_features(k,file))

In [9]:
# TODO SAVE THIS
display(pd.DataFrame(listOfSeries))
display(link_df.set_index('Track ID'))

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
0             1.690592  1.104673  0.677604  2.151633  1.373984  2.003529   
1             3.117720  2.031555  1.231667 -0.278328 -0.440500 -0.082167   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
0           1.340544  1.258241  1.304423  0.912962    ...     0.053636   
1           0.524765  0.300564  0.025170 -0.104082    ...     0.086045   

feature                               zcr                                     \
statistics                       kurtosis       max      mean    median  min   
number            05        06         01        01        01        01   01   
0           0.014008  0.014318   5.169193  0.470215  0.134251  0.135742  0.0   
1           0.018366  0.017954  12.538830  0.304199  0.060460  0.058105  0.0   

feature                         
statistics      skew       std  
number            01        01  
0           0.728773  0.044773  
1           2.462758  0.029718  

[2 rows x 518 columns]

,Song name
Track ID,
-1,IOT
-2,Tenacious D
